## This notebook ranks the employees compared to their colleagues

Imports

In [3]:
#Import data manipulation libraries
import pandas as pd
#Import statistics functions
from scipy import stats

Read in the data

In [4]:
df_dirty = pd.read_csv('./data/employee_churn_data.csv')
df_dirty.head()

,department,promoted,review,projects,salary,tenure,satisfaction,bonus,avg_hrs_month,left
0,operations,0,0.577569,3,low,5.0,0.626759,0,180.866070,no
1,operations,0,0.751900,3,medium,6.0,0.443679,0,182.708149,no
2,support,0,0.722548,3,medium,6.0,0.446823,0,184.416084,no
3,logistics,0,0.675158,4,high,8.0,0.440139,0,188.707545,no
4,sales,0,0.676203,3,high,5.0,0.577607,1,179.821083,no


Clean the data

In [5]:
df = df_dirty.copy()
#Capitalize features for better readibility
df.department = df.department.apply(lambda x: x.title() if x != "IT" else x)
df.salary = df.salary.apply(lambda x: x.title())
#Turn promoted and bonus into binary categorical data
df.promoted = df.promoted.apply(lambda x: ("Not Promoted", "Promoted")[x])
df.bonus = df.bonus.apply(lambda x: ("No Bonus", "Bonus")[x])
#Rename the `left` feature for better readibility
df.left = df.left.apply(lambda x: "Churned" if x == "yes" else "Stayed")
#Tenure is in years, so the data type should be int
df.tenure = df.tenure.astype("int64")

#Add an employee ID
df = df.reset_index().rename(columns={"index": "emp_ID"})

df.head()

,emp_ID,department,promoted,review,projects,salary,tenure,satisfaction,bonus,avg_hrs_month,left
0,0,Operations,Not Promoted,0.577569,3,Low,5,0.626759,No Bonus,180.866070,Stayed
1,1,Operations,Not Promoted,0.751900,3,Medium,6,0.443679,No Bonus,182.708149,Stayed
2,2,Support,Not Promoted,0.722548,3,Medium,6,0.446823,No Bonus,184.416084,Stayed
3,3,Logistics,Not Promoted,0.675158,4,High,8,0.440139,No Bonus,188.707545,Stayed
4,4,Sales,Not Promoted,0.676203,3,High,5,0.577607,Bonus,179.821083,Stayed


In [6]:
df.to_csv("data/employee_churn.csv")

In [99]:
#Construct a dictionary of department-salary groups of employees for quick access
department_salary_dfs = {}
departments = df.department.unique()
salaries = df.salary.unique()
for dep in departments:
    department_salary_dfs[dep] = {
        sal: df[(df.department == dep) & (df.salary == sal)] for sal in salaries
    }


def get_ranking(row, feature):
    #Get the employees in the same department-salary group
    colleagues = department_salary_dfs[row.department][row.salary]
    percentile = stats.percentileofscore(colleagues[feature], row[feature])
    if percentile > 50:
        #Convert percentile to a ranking in 10% chunks
        rank = 100 - percentile
        rank_type = "Top"
    else:
        #Convert percentile to a ranking in 10% chunks
        rank = percentile
        rank_type = "Bottom"
    #Round the rank type to be in chunks of 10%
    rank = round(rank, -1)
    #If the rank is 0%, it means it is in the top or botom 5% of employees
    rank = 5 if rank == 0 else rank

    return f"{rank_type} {int(rank)}%" 

In [101]:
#Features that determine the employee's value
employee_value_feats = ["review", "projects", "tenure", "satisfaction", "avg_hrs_month"]
#Create a data frame to rank the employee's performance as compared to their peers
employee_rankings_dict = {
    feat: df.apply(lambda row: get_ranking(row, feat), axis=1) for feat in employee_value_feats
}
#Add the employee ID so we know which employee is being ranked
employee_rankings_dict["emp_ID"] = df.emp_ID
#Convert to a data frame
employee_rankings = pd.DataFrame(employee_rankings_dict)

employee_rankings.head()

,review,projects,tenure,satisfaction,avg_hrs_month,emp_ID
0,Bottom 20%,Bottom 40%,Bottom 10%,Top 30%,Bottom 20%,0
1,Top 10%,Bottom 40%,Bottom 40%,Bottom 40%,Bottom 30%,1
2,Top 20%,Bottom 40%,Bottom 40%,Bottom 40%,Bottom 50%,2
3,Top 40%,Top 10%,Top 20%,Top 50%,Top 20%,3
4,Top 30%,Bottom 40%,Bottom 20%,Top 30%,Bottom 10%,4


Save the rankings

In [71]:
employee_rankings.to_csv("data/employee rankings 2.csv")